## Задача 0 (Свахи Круглого Стола)
> К вам в гости пришли $n$ мужчин и $n$ девушек.
> Вы решаете их посадит за круглый стол.
>
> С какой вероятностью рядом с каждым мужчиной будет сидеть ровно одна женщина?

In [ ]:
import random
from math import comb, factorial

def monte_carlo_round_table(n, trials=500_000):
  total = 2 * n
  success = 0
  for _ in range(trials):
    "0 = мужчина, 1 = женщина"
    seats = [0] * n + [1] * n
    random.shuffle(seats)
    "Проверяем: у каждого мужчины оба соседа — женщины?"
    ok = True
    for i in range(total):
      if seats[i] == 0:  # мужчина
        left = seats[(i - 1) % total]
        right = seats[(i + 1) % total]
        if left == 0 or right == 0:
          ok = False
          break
    if ok:
      success += 1
  return success / trials

def theoretical_probability(n):
  "P = 2 / C(2n, n)"
  return 2 / comb(2 * n, n)

n = 4

prob_mc = monte_carlo_round_table(n)
prob_theor = theoretical_probability(n)

print(f"Теоретическая вероятность: {prob_theor:.6f}")
print(f"Оценка Монте-Карло:        {prob_mc:.6f}")
print(f"Абсолютная ошибка:         {abs(prob_mc - prob_theor):.6f}")


Теоретическая вероятность: 0.028571
Оценка Монте-Карло:        0.028624
Абсолютная ошибка:         0.000053


# Геометрическая вероятность

> Рассмотрим числовую прямую и отрезок $[a, b] = \Omega$. Пусть случайным образом выбирается точка из этого отрезка. Это означает, что любая точка отрезка $[a, b]$ может появиться в результате эксперимента с равными шансами.
>
> Зададим вероятностную меру для любого множества $A \in \mathcal{F}_{[a,b]}$, где $\mathcal{F}_{[a,b]}$ — множество подмножеств отрезка $[a,b]$, для которых можно определить "длину". Вероятностная мера, называемая геометрической вероятностью, может быть определена следующим образом:
>
>\begin{equation*}
P(A) = \frac{\mu(A)}{\mu[a,b]} = \frac{\mu(A)}{b - a},
\end{equation*}
>
> где через $\mu(A)$ обозначена "длина" множества $A$.


> На плоскости у множества можно определить площадь. А в трёхмерном пространстве -- объём. Множества, для которых можно корректно ввести такие понятия назовём *измеримыми*.

> В более общем случае геометрическая вероятность определяется аналогично. В качестве множества элементарных событий рассмотрим некоторую измеримую область $\Omega \subset \mathbb{R}^k$. В качестве множества событий нельзя рассматривать множество всех подмножеств множества $\Omega$, так как не все подмножества множества $\Omega$ измеримы. Поэтому за события мы будем принимать только измеримые множества. Через $\mu(\Omega)$ обозначим меру в $\mathbb{R}^k$ множества $\Omega$. Используя принцип геометрической вероятности по отношению к мере $\mu$, можно определить вероятностную меру для любого измеримого множества $A \subset \Omega \subset \mathbb{R}^k$ следующим образом:
>
> \begin{equation*}
P(A) = \frac{\mu(A)}{\mu(\Omega)}.
\end{equation*}
>
> Последнее равенство принято называть определением геометрической вероятности.

## Задача 1 (Сигнал)

> В интервале времени $[0, T]$ в случайный момент $u$ появляется сигнал длительности $\Delta$.
> Приёмник включается в случайный момент $v \in [0, T]$ на время $t$.
> Предположим, что точка $(u, v)$ распределена равномерно на квадрате $[0, T]^2$
>
> Какова вероятность обнаружения сигнала?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

def detection_condition(u, v, Delta, t):
    "Проверка, пересекаются ли интервалы [u, u+Delta] и [v, v+t]."
    return not (v > u + Delta or u > v + t)

def exact_probability(T, Delta, t):
    if Delta <= 0 or t <= 0:
        return 0.0

    "Подынтегральная функция: длина отрезка u для фиксированного v"
    def integrand(v):
        u_min = max(0, v - t)
        u_max = min(T, v + Delta)
        return max(0, u_max - u_min)

    area, _ = quad(integrand, 0, T, limit=100)
    return area / (T * T)

def simulate(T, Delta, t, n_trials=10**6, seed=42):
    np.random.seed(seed)
    u = np.random.uniform(0, T, n_trials)
    v = np.random.uniform(0, T, n_trials)

    detected = (u <= v + t) & (v <= u + Delta)
    prob_estimate = np.mean(detected)

    prob_exact = exact_probability(T, Delta, t)
    return prob_estimate, prob_exact

T = 10.0
Delta = 2.0
t = 3.0
N = 1_000_000

print(f"Параметры: T = {T}, Δ = {Delta}, t = {t}")

prob_est, prob_ex = simulate(T, Delta, t, N)
print(f"Оценка вероятности (Монте-Карло): {prob_est:.6f}")
print(f"Точная вероятность:               {prob_ex:.6f}")
print(f"Абсолютная ошибка:                {abs(prob_est - prob_ex):.6f}")

Параметры: T = 10.0, Δ = 2.0, t = 3.0
Оценка вероятности (Монте-Карло): 0.435935
Точная вероятность:               0.435000
Абсолютная ошибка:                0.000935


## Задача 2 (Трамвай)

> Пассажир может воспользоваться трамваями двух маршрутов, следующих с интервалами $T_1$, $T_2$.  
> Момент прихода пассажира определяет на отрезках $[0, T_1]$, $[0, T_2]$ числа $u$ и $v$, равные временам, оставшимся до прихода трамвая соответствующего маршрута.  
>
> Предполагая, что точка $(u, v)$ равномерно распределена на $\Omega = \{(u, v): 0 \leq u \leq T_1,\ 0 \leq v \leq T_2\}$, найти вероятность того, что пассажир, пришедший на остановку, будет ждать не дольше $t$ ($0 < t < \min(T_1, T_2)$).

In [ ]:
import numpy as np

def monte_carlo_waiting_probability(T1, T2, t, num_simulations=1_000_000):
    "Генерируем u и v из равномерных распределений"
    u = np.random.uniform(0, T1, num_simulations)
    v = np.random.uniform(0, T2, num_simulations)
    "Время ожидания – минимум из u и v"
    wait_time = np.minimum(u, v)

    "Оценка вероятности"
    prob_estimate = np.mean(wait_time <= t)

    return prob_estimate

def theoretical_probability(T1, T2, t):
    "Аналитическая вероятность для случая 0 < t < min(T1,T2)"
    return 1 - ((T1 - t) / T1) * ((T2 - t) / T2)


T1, T2 = 10.0, 15.0
t = 5.0

prob_mc = monte_carlo_waiting_probability(T1, T2, t, num_simulations=2_000_000)
prob_theor = theoretical_probability(T1, T2, t)

print(f"Параметры: T1 = {T1}, T2 = {T2}, t = {t}")
print(f"Теоретическая вероятность: {prob_theor:.6f}")
print(f"Оценка Монте-Карло:        {prob_mc:.6f}")
print(f"Абсолютная ошибка:         {abs(prob_mc - prob_theor):.6f}")

Параметры: T1 = 10.0, T2 = 15.0, t = 5.0
Теоретическая вероятность: 0.666667
Оценка Монте-Карло:        0.666708
Абсолютная ошибка:         0.000041


## Задача 3 (Монетка и паркет)

> На паркет, составленный из правильных $k$-угольников со стороной $a$, случайно бросается монета радиуса $r$.
> Найти вероятность того, что упавшая монета не задевает границу ни одного из $k$-угольников паркета для:  
> 1. $k = 3$
> 2. $k = 4$


$$
P = \left(1 - \frac{2r}{a\sqrt{3}}\right)^2, \quad r \le \frac{a\sqrt{3}}{6}.
$$


## Задача 4 (Задача Бертрана)
> Рассматривается окружность радиуса $R$, в которую вписан равносторонний треугольник.
> Случайным образом в данной окружности выбирается хорда.
>
> Какова вероятность того, что длина хорды окажется больше стороны равностороннего треугольника, вписанного в эту окружность?

## Задача 5 (Определённый интеграл)

> Дана положительная ограниченная функция $f(x)$.
>
> Как приблизительно вычислить значение определённого интеграла $\int_a^b f(x) dx$?

# Деп

> Это игра для двух игроков. Игроки играют друг против друга.
>
> Каждый ($i$-ый) игрок делает следующее: выбирает место, где он встанет ($p_i$) и куда будет стрелять ($t_i$).
> Место идентифицируется числом от 1 до 1000.
>
> Цель: не быть убитым и попасть рядом с местом, где стоит противник.
>
> Считается, что игрок $i$ убит, если выстрел $j$-ого игрока попал рядом: $|p_i - t_j| < 500$.
> Матрица выигрыша следующая:
>
> |             | Попал    | Промахнулся |
> | :---------  | :------: | ----:       |
> | Попал       |   -1/-1  | 0/2         |
> | Промахнулся |   2/0    | 1/1         |
>